In [ ]:
#sqrt(1+sqr(x)*sqr(sin(sqr(x)))) where -1<=x<=1

# Importing

In [199]:
import numpy as np
import math
from math import sqrt
from math import pow
from math import sin
from math import cos
from scipy.interpolate import lagrange
eps = 0.0000001

## Segment splitting

In [72]:
def split(a, b, n): # a & b is a segment, n is an amount of fragments
    lengthOfSegment = b-a
    lengthOfFragment = lengthOfSegment / (n-1) #last point should be equal to b
    splitPoints = np.zeros(n) 
    for i in range(n):
        splitPoints[i] = a + i * lengthOfFragment
    return splitPoints
  ################## Example      
split(1, 20, 5)

array([ 1.  ,  5.75, 10.5 , 15.25, 20.  ])

## Here's my function hardcoded

In [73]:
def calcFunValue(value): #Calculates values of calcFunValue
    x = value
    a = -1
    b = 1
    if (value < a) or (value > b):
        return None
    return sqrt(1 + pow(x, 2) * pow(sin(pow(x, 2)), 2))
################## Example
print(calcFunValue(1.1))

None


# Interpolation

## Calculates values of function hardcoded above into nodes values

In [74]:
def valuesInPonts(points): #Calculates values of calcFunValue in points array
    values = np.zeros(len(points))
    for i in range(len(points)):
        values[i] = calcFunValue(points[i])
    return values

## My own lagrange implementation

In [ ]:
#Evaluates te i-th Lagrange polynomial at x
def my_lagrange (x ,i , xm ):
    y=1.
    for j in range ( len(xm) ):
        if i != j:
            y*=( x-xm[j])/( xm[i]-xm[j])
    return y

#Interpolation
def lagrangeInterpolation (x , xm , ym ): 
    n=len( xm )
    lagrpoly = [ my_lagrange (x ,i , xm ) for i in range ( n )]
    y = np.dot( ym , lagrpoly ) #Vector's multiplication
    return y

def interpolateFunction(new_x, xm, ym):
    new_y = [lagrangeInterpolation(x, xm, ym) for x in new_x]
    return new_y

## Newtone interpolation

In [93]:
def _poly_newton_coefficient(x,y):
    """
    x: list or np array contanining x data points
    y: list or np array contanining y data points
    """

    m = len(x)

    x = np.copy(x)
    a = np.copy(y)
    for k in range(1,m):
        a[k:m] = (a[k:m] - a[k-1])/(x[k:m] - x[k-1])

    return a

def newton_polynomial(x_data, y_data, x):
    """
    x_data: data points at x
    y_data: data points at y
    x: evaluation points
    """
    a = _poly_newton_coefficient(x_data, y_data)
    n = len(x_data) - 1 # Degree of polynomial
    p = a[n]
    for k in range(1,n+1):
        p = a[n-k] + (x -x_data[n-k])*p
    return p

## Second lab simple print

In [104]:
def printTable(x, y, y_newt, y_lagr):
    print("x   ", "f(x)   ", "P(x)   ", "L(x)")
    for i in range(len(x)):
        print(x[i], y[i], y_newt[i], y_lagr[i])

        


## Third lab print

In [144]:
def printTable3(n):
    i0 = left_rectangles_rule(calcFunValue, -1, 1, n)
    i1 = trapezoid_rule(calcFunValue, -1, 1, n)
    i2 = simpson_rule(calcFunValue, -1, 1, n)
    print("N=", n, " I0=", i0, " I1=", i1, " I2=", i2)

# Integration

In [124]:
def integrate(f, a, b, p, method):
    
    # Spliting 
    # p means precision
    
    n = 2
    result1 = method(f, a, b, n)
    n *= 2
    result2 = method(f, a, b, n)
    
    while abs(result1 - result2) > p:
        n *= 2
        result1 = method(f, a, b, n)
        n *= 2
        result2 = method(f, a, b, n)
    return result2

## Integration by rectangles (left)

In [130]:
def left_rectangles_rule(f, a, b, n):
    offset = (b-a)/float(n)  #Calculate offset with current N
    total = sum([f((a + (k * offset))) for k in range(0, n)])   #Sum of f(x) from offsetted values 
    result = offset * total   #Multiply rusult to iffset value (look at lefr rectangles formula)
    return result

## Trapezoid integration

In [131]:
def trapezoid_rule(f, a, b, n):
    dx = 1.0 * (b - a) / n # Calculate offset
    sum = 0.5 * (f(a) + f(b))
    for i in range(1, n):
        sum += f(a + i * dx)

    return sum * dx

## Simpson integration

In [146]:
def simpson_rule(f, a, b, n):
    
    if n % 2 == 1:
        n += 1  #Even parabols number only
    dx = (b - a) / n
    sum = (f(a) + 4 * f(a + dx) + f(b))
    for i in range(1, (int)(n / 2)):
        sum += 2 * f(a + (2 * i) * dx) + 4 * f(a + (2 * i + 1) * dx)
    return sum * dx / 3

# Cauchy issue solving

In [217]:
def euler_method(function, n, xlist, ylist):
    h = 1/n
    
    for i in range(1, n):
        ylist[i] = ylist[i-1] + h * function(xlist[i-1], ylist[i-1])
    return ylist

In [197]:
def f3(x, y):
    return y * cos(x)

def f3_true(x):
    return math.e ** math.sin(x) # exact solution

#Known solution
x0 = 0
y0 = 1

In [230]:
def printTable4(xlist, ylist_tilda, ylist):
    for i in range(len(xlist)):
        print(xlist[i], ylist_tilda[i], ylist[i], math.abs(ylist[i] - ylist_tilda[i]))

# /////////////////////////////////////////////////Examples//////////////////////////////////////////////////////////

## Lab1 & Lab2

In [150]:
# Obtain x values on the vector. The last one is 'M'
x_new_values = split(-1, 1, 6) 

#Calculate true y values
y_new_true_values = valuesInPonts(x_new_values) 

# Calculate interpolated values based on lagrange
y_new_Lagrange_values = interpolateFunction(x_new_values, x_values, y_values) 

# Calculate interpolated values based on Newtonr
y_new_Newtone_values = [newton_polynomial(x_values, y_values, x) for x in x_new_values] 

#Printing
printTable(x_new_values, y_new_true_values, y_new_Newtone_values, y_new_Lagrange_values)

x    f(x)    P(x)    L(x)
-1.0 1.306932828523934 1.306932828523934 1.306932828523934
-0.6 1.0220934246704843 1.0864328224313473 1.0864328224313473
-0.19999999999999996 1.0000319824255361 0.9761828193850539 0.9761828193850539
0.20000000000000018 1.0000319824255361 0.9761828193850539 0.9761828193850537
0.6000000000000001 1.0220934246704843 1.0864328224313473 1.0864328224313473
1.0 1.306932828523934 1.306932828523934 1.306932828523934


## Lab3

In [134]:
integrate(calcFunValue, -1, 1, eps, left_rectangles_rule)

2.106020707147699

In [135]:
integrate(calcFunValue, -1, 1, eps, trapezoid_rule)

2.106020707147699

In [140]:
integrate(calcFunValue, -1, 1, eps, simpson_rule)

2.106019721771366

In [147]:
printTable3(1000)

N= 1000  I0= 2.1060207453674433  I1= 2.1060207453674433  I2= 2.1060199203421273


## Lab 4

In [231]:
n = 1000
h = 1/n

a,b = x0, 1

x_values = split(a, b, n)
y_values = np.zeros(n)
y_values[0] = y0

y_values_tilda = euler_method(f3, n, x_values, y_values)

y_values = [f3_true(x) for x in x_values]

printTable4(x_values, y_values_tilda, y_values)

AttributeError: module 'math' has no attribute 'abs'